In [53]:
import pandas as pd
import pickle as pkl
import numpy as np
from tqdm import tqdm

from pytorch_transformers import BertTokenizer, BertForMaskedLM, BertForSequenceClassification, BertConfig
from util import *

# get gt based on sst labels

In [25]:
sents_arr = np.loadtxt('data/sentiment_labels.txt', dtype=str, delimiter='|')
id_to_sent = {sents_arr[i, 0]: float(sents_arr[i, 1]) for i in range(1, sents_arr.shape[0])}

phrases = open('data/dictionary.txt', 'r').read().split('\n') #[:10000]
phrase_to_id = {}
for line in phrases:
    try:
        s, phrase_id = line.split('|')
        if all([c.isalpha() for c in s]):
            phrase_to_id[s.lower()] = phrase_id
    except:
        pass
    
phrase_to_sent = {
    phrase: id_to_sent[phrase_to_id[phrase]] for phrase in phrase_to_id if phrase_to_id[phrase] in id_to_sent
}

In [ ]:
scores = pkl.loads
final_scores_iid = [np.mean(scores_iid[k]) for k in scores_iid]